In [ ]:
from ray import tune
from ultralytics import YOLO
import torch
import os

"""
BEVDetNet params:
epochs: 50, 60, 80, 120
batch_size: 8, 16, 24, 32, 48
imgsz: 640x640x3
optimizer: Adam
loss weights: 0.95, 0.98, 1
"""

# Set expandable_segments to reduce memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check for GPU use
print('Use GPU:', torch.cuda.is_available())

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared.\n")

tune_config = {
    "epochs": tune.choice([50, 60, 80, 100, 120]),
    "batch": tune.choice([4, 8, 16, 24, 32, 48]),
    "optimizer": tune.choice(["SGD", "Adam", "AdamW", "NAdam", "RAdam", "RMSProp", "auto"]),
    "lr0": tune.uniform(1e-5, 1e-1),  # Initial learning rate , (1e-5, 1e-1)
    "lrf": tune.uniform(1e-5, 1e-1),  # Final learning rate (0.01, 1.0)
    "momentum": tune.choice([0.6, 0.7, 0.8, 0.837, 0.9, 0.937]), # (0.6, 0.98)
    "weight_decay": tune.choice([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006]), # (0.0, 0.001)
    "warmup_epochs": tune.choice([2.0, 3.0, 4.0, 5.0, 6.0, 7.0]), # (0.0, 5.0)
    "warmup_momentum": tune.choice([0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), # (0.0, 0.95)
    "warmup_bias_lr": tune.choice([0.1, 0.2, 0.3, 0.4, 0.5, 0.6]), 
    "box": tune.choice([6.0, 6.5, 7.0, 7.5, 8.0, 8.5]),  # Box loss weight (0.02, 0.2)
    "cls": tune.choice([0.3, 0.5, 0.7, 1.0, 1.2, 1.5]),  # Class loss weight (0.2, 4.0)
    "dfl": tune.choice([0.5, 1.0, 1.2, 1.5, 1.8, 2.0]),  # DFL loss weight
    "nbs": tune.choice([4, 8, 16, 24, 32, 48, 64]),  # Nominal batch size
}

# Build a new model from scratch and check model specific values
print('--Check YOLO intern values for custom purposes--')
# Build a new model from scratch
model = YOLO('yolov8s-obb.yaml', 'obb', verbose=False)

# Train the model with hyperparameters from Ray Tune
result_grid = model.tune(
    data='/home/rlab10/ultralytics_yolov8-obb_ob_kitti/ultralytics/cfg/datasets/kitti_bev.yaml', use_ray=True, space=tune_config,
    grace_period=10, gpu_per_trial=1, iterations=10,
    # **train_args
    time=None, patience=10, imgsz=640, save=True, save_period=25, cache=True, device=0, workers=4, project='kitti_bev_yolo_rayt', 
    name='tune_experiment', exist_ok=False, pretrained=False, seed=0, deterministic=False, single_cls=False, classes=None, rect=False, 
    cos_lr=False, close_mosaic=0, resume=True, amp=False, fraction=1.0, profile=False, freeze=None,  pose=0.0, kobj=0.0,  overlap_mask=False, 
    mask_ratio=0, dropout=0.0, val=True, plots=False,
    # augmentation args                      
    hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, degrees=0.0, translate=0.0, scale=0.0, shear=0.0, perspective=0.0,flipud=0.0, fliplr=0.0, bgr=0.0, mosaic=0.0, 
    mixup=0.0, copy_paste=0.0, copy_paste_mode='flip', auto_augment=None,erasing=0.0, crop_fraction=0.0)


In [ ]:
from ray import tune
from ray.air import RunConfig
from ray.tune import Tuner
from ultralytics import YOLO

# Define the training function
def train_yolo(config):
    model = YOLO('yolov8s-obb.yaml', 'obb', verbose=False)
    results = model.train(**config)
    return results

# Define the search space
tune_config = {
    "lr0": tune.uniform(1e-5, 1e-1),
    # Add other hyperparameters here
}

# Configure Ray Tune
tuner = Tuner(
    train_yolo,
    param_space=tune_config,
    run_config=RunConfig(
        name="yolo_tune_experiment",
        storage_path="~/ray_results",  # Use storage_path instead of local_dir
    ),
)

# Run the tuning
results = tuner.fit()